In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from statsmodels.stats.multitest import multipletests
import warnings
from sklearn.decomposition import FactorAnalysis
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv
from pandas import set_option
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
from imblearn.pipeline import Pipeline, make_pipeline
from skrebate import ReliefF
from sklearn.feature_selection import SelectFromModel
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.feature_selection import f_classif
from skrebate import ReliefF, SURF, MultiSURF
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import VotingClassifier
from skrebate import SURF
from skrebate import SURFstar
from skrebate import MultiSURF
from skrebate import MultiSURFstar
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from scipy.stats import uniform, randint
from scipy.stats import randint as sp_randint
import warnings
import time
import os
from imblearn.over_sampling import BorderlineSMOTE, ADASYN, KMeansSMOTE
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

# get the start time
st = time.time()
file_name = 'cancerprediction_rads_T0_lasso'
#Load the data
main_dir= "/home/ulaval.ca/lesee/projects/Project-NLST/"


df_rad_features_train = pd.read_csv(os.path.join(main_dir,'results/radiomics-ComBat_harmonized/cohort1_T0_rads_combat_kerneltype_notdropped.csv'))

df_rad_features_test = pd.read_csv(os.path.join(main_dir,'results/radiomics-ComBat_harmonized/cohort2_T0_rads_combat_kerneltype_notdropped.csv'))

df_labels_train = pd.read_csv(os.path.join(main_dir,'data/data_batch-clinical-labels/Cohort1_labels.csv'))
df_labels_test = pd.read_csv(os.path.join(main_dir,'data/data_batch-clinical-labels/Cohort2_labels.csv'))

# perform an inner join on the 'PatientID' column
data_train = pd.merge(df_rad_features_train, df_labels_train, on='PatientID', how='inner')
data_test = pd.merge(df_rad_features_test, df_labels_test, on='PatientID', how='inner')

print(np.shape(data_train))
print(np.shape(data_test))

(254, 1111)
(208, 1111)


In [2]:
#save the joined DataFrame to a new CSV file
#joined_df.to_csv('joined_file.csv', index=False)


#data_train = pd.read_excel(os.path.join(main_dir,'data/radiomicsfeatures_kheops-NLST-Dmitrii-Cohort1_Laptop_v2-cleaned-T0-ML.xlsx'))
#data_test = pd.read_excel(os.path.join(main_dir,'data/radiomicsfeatures_kheops-NLST-Dmitrii-Cohort2_Laptop_v2-cleaned-T0-ML.xlsx'))

# Check if the folder exists, and create it if it doesn't
if not os.path.exists(os.path.join(main_dir, 'results')):
    os.makedirs(os.path.join(main_dir, 'results'))
if not os.path.exists(os.path.join(main_dir, 'results/'+file_name)):
    os.makedirs(os.path.join(main_dir, 'results/'+file_name))
    
# Preprocess the data

# Explicitly create a copy of the data
data_train_copy = data_train.copy()
data_test_copy = data_test.copy()

# Encoding with map
data_train_copy['label_encoded'] = data_train_copy['Label'].map({'benign': 0, 'malignant': 1})
data_test_copy['label_encoded'] = data_test_copy['Label'].map({'benign': 0, 'malignant': 1})

# Set the target variable as recurrence for stage 1 and stage 2
target_train= data_train_copy['label_encoded'].copy()
target_test= data_test_copy['label_encoded'].copy()

# Set the features as the radiomic features
features_train = data_train_copy.drop(columns=['label_encoded','Label', 'PatientID'])
features_test = data_test_copy.drop(columns=['label_encoded','Label', 'PatientID'])

print("number of features for training set = ", np.shape(features_train))
print("number of features for test set = ", np.shape(features_test))

#Remove constant radiomic features
constant_features = features_train.columns[features_train.nunique() == 1]
features_train.drop(constant_features, axis=1, inplace=True)
features_test.drop(constant_features, axis=1, inplace=True)
print("number of non-constant features in training data = ", np.shape(features_train))
print("number of non-constant features in test data = ", np.shape(features_test))

# Store the feature names
feature_names = features_train.columns.tolist()
print("Initial number of features = " , len(feature_names))


# Split the data into training and testing sets
X_train = features_train
X_validation =  features_test
y_train = target_train
y_validation = target_test

X_train_init = X_train.copy()
X_validation_init = X_validation.copy()

# Scale the features of the balanced training set
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)

# Separate positive and negative cases in the training set
positive_cases_X_train = X_train[(y_train == 1)]
negative_cases_X_train = X_train[(y_train == 0)]

# Separate positive and negative cases in the test set
positive_cases_X_test = X_validation[(y_validation == 1)]
negative_cases_X_test = X_validation[(y_validation == 0)]


print("number of positive_train = ", len(positive_cases_X_train))
print("number of negative_train = ", len(negative_cases_X_train))

print("number of positive_test = ", len(positive_cases_X_test))
print("number of negative_test = ", len(negative_cases_X_test))

number of features for training set =  (254, 1109)
number of features for test set =  (208, 1109)
number of non-constant features in training data =  (254, 1109)
number of non-constant features in test data =  (208, 1109)
Initial number of features =  1109
number of positive_train =  86
number of negative_train =  168
number of positive_test =  75
number of negative_test =  133


In [3]:
# Assuming feature_names is defined and contains the names of your features
# Assuming X_train_scaled, y_train, X_validation_scaled, and y_validation are already defined

# Define the cross-validation strategy
cv = RepeatedKFold(n_splits=5, n_repeats=10)

# For classification, define an appropriate scorer. For example, using accuracy:
def my_scorer(y_test, y_pred):
    score = roc_auc_score(y_test, y_pred)
    return score

my_func = make_scorer(my_scorer, greater_is_better=True)

# Use Logistic Regression with L1 penalty for feature selection
lasso_logistic = LogisticRegression(penalty='l1', solver='liblinear', max_iter=5000)

param_grid = {'C': [1 / (i * 0.5) for i in range(1, 100)]}  # Note: C is the inverse of alpha

grid_search = GridSearchCV(lasso_logistic, param_grid, cv=cv, scoring=my_func)
grid_search.fit(X_train_scaled, y_train)

best_C = grid_search.best_params_['C']
lasso_logistic_best = LogisticRegression(penalty='l1', solver='liblinear', C=best_C, max_iter=5000)
lasso_logistic_best.fit(X_train_scaled, y_train)

coefficients = lasso_logistic_best.coef_[0]  # Get the coefficients of the fitted model
zero_coefficient_indices = list(np.where(coefficients == 0)[0])

X_train_scaled_lasso = X_train_scaled[:, ~np.isin(np.arange(X_train_scaled.shape[1]), zero_coefficient_indices)]
X_validation_scaled_lasso = X_validation_scaled[:, ~np.isin(np.arange(X_validation_scaled.shape[1]), zero_coefficient_indices)]

# Get the indices of the selected features after Lasso-based feature selection
selected_feature_indices_after_lasso = np.where(~np.isin(np.arange(X_train_scaled.shape[1]), zero_coefficient_indices))[0]

# Get the names of the features after Lasso-based feature selection
selected_feature_names_after_lasso = [feature_names[idx] for idx in selected_feature_indices_after_lasso]

final_selected_features = selected_feature_names_after_lasso 





X_train_scaled_lasso_df = pd.DataFrame(X_train_scaled_lasso, columns=final_selected_features)
y_train_df = pd.DataFrame({'y_train': y_train})

X_test_scaled_lasso_df = pd.DataFrame(X_validation_scaled_lasso, columns=final_selected_features)
y_test_df = pd.DataFrame({'y_test': y_validation})

# Concatenate the DataFrames horizontally (side by side)
combined_df_train = pd.concat([X_train_scaled_lasso_df, y_train_df], axis=1)
combined_df_test = pd.concat([X_test_scaled_lasso_df, y_test_df], axis=1)

# Save the combined DataFrame to a CSV file
combined_df_train.to_csv(os.path.join(main_dir,'data/Training-Lasso-scaled-combat-kerneltype_notdropped-T0.csv'), index=False, float_format='%.7f')

combined_df_test.to_csv(os.path.join(main_dir,'data/Test-Lasso-scaled-combat-kerneltype_notdropped-T0.csv'), index=False, float_format='%.7f')




print("Shape of X_test after Lasso = ", np.shape(X_validation_scaled_lasso))
print("Shape of X_train after Lasso = ", np.shape(X_train_scaled_lasso))
print("Shape of y_test after Lasso = ", np.shape(y_validation))
print("No. of features after Lasso = ", len(final_selected_features))

Shape of X_test after Lasso =  (208, 23)
Shape of X_train after Lasso =  (254, 23)
Shape of y_test after Lasso =  (208,)
No. of features after Lasso =  23
